In [14]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMinimize, LpStatus, lpSum

df_hubs = pd.read_csv("hubs.csv")
hubs = {
    row["City"].strip(): {
        "current_tons": int(row["Current tons"]),
        "capacity": int(row["Capacity (monthly tons)"])
    }
    for _, row in df_hubs.iterrows()
}

df_focus = pd.read_csv("focus_cities.csv")
focus_cities = {
    row["City"].strip(): {
        "airport": row["Airport"],
        "capacity": int(row["Capacity (monthly tons)"])
    }
    for _, row in df_focus.iterrows()
}

df_centers = pd.read_csv("centers.csv")
centers = {}
for _, row in df_centers.iterrows():
    country = row["Country"].strip()
    city = row["City"].strip()
    demand = int(row["Demand (monthly tons)"])
    centers.setdefault(country, {})[city] = demand

In [15]:
df_costs = pd.read_csv("costs.csv")

df_costs.columns = df_costs.columns.str.strip().str.replace('\ufeff', '')

df_costs = df_costs.replace("N/A", pd.NA).replace("", pd.NA)

costs = {}
for _, row in df_costs.iterrows():
    destination = row["Suppliers"]
    costs[destination] = {}
    for supplier in df_costs.columns[1:]:
        val = row[supplier]
        costs[destination][supplier] = float(val) if pd.notna(val) else None

In [16]:
def flatten_centers(centers):
    """Convert centers from {country: {city: demand}} to {city: demand}."""
    return {city: demand for country, cities in centers.items() for city, demand in cities.items()}

flattened_centers = flatten_centers(centers)

DEFAULT_COST = 10000

def get_cost(src, dest, cost_data, default_cost=DEFAULT_COST):
    """Return cost from supplier src to destination dest, or default if missing."""
    return cost_data.get(dest, {}).get(src, default_cost)

In [17]:
prob = LpProblem("Cargo_Optimization", LpMinimize)

x = LpVariable.dicts(
    "shipment_from_hub_to_focus",
    (hubs.keys(), focus_cities.keys()),
    lowBound=0,
    cat="Continuous"
)

y = LpVariable.dicts(
    "shipment_from_focus_to_center",
    (focus_cities.keys(), flattened_centers.keys()),
    lowBound=0,
    cat="Continuous"
)

prob += (
    lpSum(
        get_cost(h, f, costs) * x[h][f]
        for h in hubs
        for f in focus_cities
        if get_cost(h, f, costs) is not None and get_cost(h, f, costs) < DEFAULT_COST
    )
    +
    lpSum(
        get_cost(f, c, costs) * y[f][c]
        for f in focus_cities
        for c in flattened_centers
        if get_cost(f, c, costs) is not None and get_cost(f, c, costs) < DEFAULT_COST
    )
), "Total_Shipping_Cost"

In [18]:
for h in hubs:
    prob += lpSum([x[h][f] for f in focus_cities]) <= hubs[h]["capacity"], f"Hub_{h}_Capacity"

for f in focus_cities:
    prob += lpSum([x[h][f] for h in hubs]) <= focus_cities[f]["capacity"], f"FocusCity_{f}_Capacity"

for city, demand in flattened_centers.items():
    prob += lpSum([y[f][city] for f in focus_cities]) == demand, f"Demand_{city}"

for f in focus_cities:
    prob += lpSum([x[h][f] for h in hubs]) == lpSum([y[f][city] for city in flattened_centers]), f"Flow_Balance_{f}"

prob.solve()

if LpStatus[prob.status] == "Optimal":
    for h in hubs:
        for f in focus_cities:
            if x[h][f].varValue > 0:
                print(f"Ship {x[h][f].varValue} tons from {h} to {f}")
    for f in focus_cities:
        for city in flattened_centers:
            if y[f][city].varValue > 0:
                print(f"Ship {y[f][city].varValue} tons from {f} to {city}")
else:
    print("No optimal solution found")

Ship 34397.0 tons from CVG to Leipzig
Ship 19000.0 tons from CVG to Hyderabad
Ship 36000.0 tons from CVG to San Bernardino
Ship 44350.0 tons from AFW to Leipzig
Ship 6500.0 tons from Leipzig to Paris
Ship 300.0 tons from Leipzig to Cologne
Ship 180.0 tons from Leipzig to Hanover
Ship 9100.0 tons from Leipzig to Bangalore
Ship 90.0 tons from Leipzig to Cagliari
Ship 185.0 tons from Leipzig to Catania
Ship 800.0 tons from Leipzig to Milan
Ship 1700.0 tons from Leipzig to Rome
Ship 170.0 tons from Leipzig to Katowice
Ship 2800.0 tons from Leipzig to Barcelona
Ship 3700.0 tons from Leipzig to Madrid
Ship 190.0 tons from Leipzig to Mobile
Ship 175.0 tons from Leipzig to Anchorage
Ship 38.0 tons from Leipzig to Fairbanks
Ship 2400.0 tons from Leipzig to Phoenix
Ship 7200.0 tons from Leipzig to Los Angeles
Ship 100.0 tons from Leipzig to Ontario
Ship 1200.0 tons from Leipzig to Riverside
Ship 1100.0 tons from Leipzig to Sacramento
Ship 1900.0 tons from Leipzig to San Francisco
Ship 240.0 tons

In [19]:
prob.solve()

for h in hubs:
    total = sum(x[h][f].varValue for f in focus_cities if x[h][f].varValue is not None)
    cap = hubs[h]["capacity"]
    status = "OK ✅" if total <= cap else "VIOLATION ⚠️"
    print(f"Hub {h}: {total:.2f} / {cap} ({status})")

Hub CVG: 89397.00 / 95650 (OK ✅)
Hub AFW: 44350.00 / 44350 (OK ✅)


In [20]:
for f in focus_cities:
    values = [x[h][f].varValue for h in hubs if x[h][f].varValue is not None]
    total_shipment_to_focus_city = sum(values) if values else 0
    
    cap = focus_cities[f]["capacity"]
    status = "OK ✅" if total_shipment_to_focus_city <= cap else "VIOLATION ⚠️"
    print(f"Focus city {f}: {total_shipment_to_focus_city:.2f} / {cap} ({status})")

Focus city Leipzig: 78747.00 / 85000 (OK ✅)
Focus city Hyderabad: 19000.00 / 19000 (OK ✅)
Focus city San Bernardino: 36000.00 / 36000 (OK ✅)


In [23]:
for city, demand in flattened_centers.items():
    values = [y[f][city].varValue for f in focus_cities if y[f][city].varValue is not None]
    total_shipment_to_center = sum(values) if values else 0
    
    if abs(total_shipment_to_center - demand) <= 1e-3:
        print(f"Center {city}: demand satisfied ✅ ({total_shipment_to_center:.2f} / {demand})")
    else:
        print(f"Center {city}: demand VIOLATION ⚠️ ({total_shipment_to_center:.2f} / {demand})")


Center Paris: demand satisfied ✅ (6500.00 / 6500)
Center Cologne: demand satisfied ✅ (640.00 / 640)
Center Hanover: demand satisfied ✅ (180.00 / 180)
Center Bangalore: demand satisfied ✅ (9100.00 / 9100)
Center Coimbatore: demand satisfied ✅ (570.00 / 570)
Center Delhi: demand satisfied ✅ (19000.00 / 19000)
Center Mumbai: demand satisfied ✅ (14800.00 / 14800)
Center Cagliari: demand satisfied ✅ (90.00 / 90)
Center Catania: demand satisfied ✅ (185.00 / 185)
Center Milan: demand satisfied ✅ (800.00 / 800)
Center Rome: demand satisfied ✅ (1700.00 / 1700)
Center Katowice: demand satisfied ✅ (170.00 / 170)
Center Barcelona: demand satisfied ✅ (2800.00 / 2800)
Center Madrid: demand satisfied ✅ (3700.00 / 3700)
Center Castle Donington: demand satisfied ✅ (30.00 / 30)
Center London: demand satisfied ✅ (6700.00 / 6700)
Center Mobile: demand satisfied ✅ (190.00 / 190)
Center Anchorage: demand satisfied ✅ (175.00 / 175)
Center Fairbanks: demand satisfied ✅ (38.00 / 38)
Center Phoenix: demand sati

In [22]:
for f in focus_cities:
    total_flow_in = sum(x[h][f].varValue for h in hubs)
    total_flow_out = sum(y[f][city].varValue for city in flattened_centers)
    if total_flow_in == total_flow_out:
        print(f"Flow balance for focus city {f} satisfied: {total_flow_in} == {total_flow_out}")
    else:
        print(f"Flow balance for focus city {f} violated: {total_flow_in} != {total_flow_out}")

Flow balance for focus city Leipzig satisfied: 78747.0 == 78747.0
Flow balance for focus city Hyderabad satisfied: 19000.0 == 19000.0
Flow balance for focus city San Bernardino satisfied: 36000.0 == 36000.0
